# Dynamic Programming
## 0/1 Knapsack

Given a list of items with their assocaited weight and value, and an overall weight limit, determine which items will provide the maximum total value while not exceeding the weight limit. The item must be taken in full e.g. you cannot take half of item one for half of its value.

To accomplish this, we define a measure T[i, j] which is the maximum value we can obtain at a weight j considering up to i items.

Given a list of objects $O: \{O_1, O_2, ... \}$ with a $(weight, value)$ tuple for each object, and an overall weight limit $w$ use this method:

1. Create a (O + 1) x (w + 1) matrix.
- The row indexes are the number of items considered. e.g. Row 0 is a row of padding, no items are considered. Row 1 considers only the first item. Row 2 considers the first 3 items. etc.
- The column indexes are the maximmum weight limit. e.g. if the overall weight limit is 3: Row 0 is a row of padding, the weight limit is 0 so no items are taken. Row 1 has a weight limit of 1, select items conforming to this limit. Row 2 has a weight limit of 2. Row 3 has a weight limit of 3.
2. Initialize padding columns to 0. This would be all values in row 0 and column 0.
3. Starting at (1, 1) we take the max value of the following 2 cases:
- Case I: We are including the ith object in our solution:   
-- If the weight of the ith object < column j then value of T[i, j] would be value of object i + T[i-1, j - weight of object i]
- Case II: We are not including the ith object in our solution   
-- The value of T[i, j] would be the same as the previous solution T[i-1, j]
4. The final solution will be at the last cell of this matrix

The tables below demonstrate the progress of the algorithm. The 3 columns added on the left side are provided to show you the associated weight and value of each item.

In [1]:
import numpy as np
import pandas as pd

def style_specific_cell(df, row, col, c):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    df_styler.iloc[row, col] = 'color: black;background-color: ' + c
    return df_styler

def style_case1_fails(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    #current
    df_styler.iloc[row, col] = 'color: black; background-color: gold'
    #one above
    df_styler.iloc[row-1, col] = 'color: black; background-color: lightgreen'
    #weight
    df_styler.iloc[row, 0] = 'color: black; background-color: firebrick; font-weight:bold'
    #value
    df_styler.iloc[row, 1] = 'color: black; background-color: firebrick; font-weight:bold'
    #column
    df_styler.iloc[0, col] = 'color: black; background-color: firebrick; font-weight:bold'
    return df_styler

def style_case1(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    #current
    df_styler.iloc[row, col] = 'color: black; background-color: gold'
    # one above
    df_styler.iloc[row-1, col] = 'color: black; background-color: firebrick'
    #weight 
    df_styler.iloc[row, 0] = 'color: black; background-color: lightgreen; font-weight:bold'
    #value 
    df_styler.iloc[row, 1] = 'color: black; background-color: lightgreen; font-weight:bold'
    #column value
    df_styler.iloc[0, col] = 'color: black; background-color: lightgreen; font-weight:bold'
    #case 1 added value
    df_styler.iloc[row-1,col-df.iloc[row, 0]] = 'color: black; background-color: lightgreen; font-weight:bold' 
    #column
    df_styler.iloc[0, col] = 'color: black; background-color: lightgreen; font-weight:bold'

    return df_styler

def style_case2(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    #current
    df_styler.iloc[row, col] = 'color: black; background-color: gold'
    # one above
    df_styler.iloc[row-1, col] = 'color: black; background-color: lightgreen'
    #weight 
    df_styler.iloc[row, 0] = 'color: black; background-color: darkorange; font-weight:bold'
    #value 
    df_styler.iloc[row, 1] = 'color: black; background-color: darkorange; font-weight:bold'
    #column val
    df_styler.iloc[0, col] = 'color: black; background-color: darkorange; font-weight:bold'
    #case 1 added value
    df_styler.iloc[row-1,col-df.iloc[row, 0]] = 'color: black; background-color: darkorange; font-weight:bold'
    #column
    df_styler.iloc[0, col] = 'color: black; background-color: darkorange; font-weight:bold'

    return df_styler

def style_traceback(df, row, col):
    df_styler = pd.DataFrame('', index = df.index, columns = df.columns)
    df_styler.iloc[row, col] = 'color: black; background-color: gold'
    x = row
    y = col
    while df.iloc[x, y] != 0:
        if df.iloc[x-1, y] == df.iloc[x,y]:
            df_styler.iloc[x-1, y] = 'color: black; background-color: gold'
            x -= 1
        else:
            df_styler.iloc[x-1, y-df.iloc[x,0]] = 'color: black; background-color: gold'
            df_styler.iloc[x,3] = 'color: black; background-color: yellowgreen'
            y -= df.iloc[x,0]
            x -= 1
    return df_styler

In [2]:
################################# DEFINE [WEIGHT, VALUE] FOR EACH OBJECT HERE #################################
objects = [['W','V'], ['-','-'], [2, 7], [5, 14], [1, 8], [6, 16], [7, 2]] #leave first 2 items
weight = 11 #OVERALL WEIGHT LIMIT
###############################################################################################################

rowName = "#"
for x in range(len(objects)-1):
        rowName += str(x)
offset = 5
df = pd.DataFrame(np.zeros((len(rowName), weight + offset))).astype(int)
df.iloc[0] = [x for x in range(-(offset - 1), weight+1)]
df.iloc[:, 0] = [x[0] for x in objects]
df.iloc[:, 1] = [x[1] for x in objects]
df.iloc[:, 2] = [' ' for x in rowName]
df.iloc[:, 3] = [x for x in rowName]

df.style.hide(axis=1).hide(axis=0)

W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,0,0,0,0,0,0,0,0,0,0
5,14,,2,0,0,0,0,0,0,0,0,0,0,0,0
1,8,,3,0,0,0,0,0,0,0,0,0,0,0,0
6,16,,4,0,0,0,0,0,0,0,0,0,0,0,0
7,2,,5,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
for i in range(2,df.shape[0]):
    for j in range(5,df.shape[1]):
        case1 = 0
        case2 = df.iloc[i-1,j]
        #check if object weight is less than column
        if df.iloc[i,0] <= df.iloc[0,j]:
            case1 = df.iloc[i,1] + df.iloc[i-1,j-df.iloc[i,0]]
        
        if df.iloc[i,0] > df.iloc[0,j]:
            df.iloc[i,j] = case2
            temp = df.style.apply(style_case1_fails, row = i, col = j, axis = None).hide(axis=1).hide(axis=0)
        elif case1 > case2:
            df.iloc[i,j] = case1
            temp = df.style.apply(style_case1, row = i, col = j, axis = None).hide(axis=1).hide(axis=0)
        else:
            df.iloc[i,j] = case2
            temp = df.style.apply(style_case2, row = i, col = j, axis = None).hide(axis=1).hide(axis=0)


        display(temp)
temp = df.style.apply(style_traceback, row = (df.shape[0]-1), col = (df.shape[1]-1), axis = None).hide(axis=1).hide(axis=0)
display(temp)




W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,0,0,0,0,0,0,0,0,0,0
5,14,,2,0,0,0,0,0,0,0,0,0,0,0,0
1,8,,3,0,0,0,0,0,0,0,0,0,0,0,0
6,16,,4,0,0,0,0,0,0,0,0,0,0,0,0
7,2,,5,0,0,0,0,0,0,0,0,0,0,0,0


W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,7,0,0,0,0,0,0,0,0,0
5,14,,2,0,0,0,0,0,0,0,0,0,0,0,0
1,8,,3,0,0,0,0,0,0,0,0,0,0,0,0
6,16,,4,0,0,0,0,0,0,0,0,0,0,0,0
7,2,,5,0,0,0,0,0,0,0,0,0,0,0,0


W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,7,7,0,0,0,0,0,0,0,0
5,14,,2,0,0,0,0,0,0,0,0,0,0,0,0
1,8,,3,0,0,0,0,0,0,0,0,0,0,0,0
6,16,,4,0,0,0,0,0,0,0,0,0,0,0,0
7,2,,5,0,0,0,0,0,0,0,0,0,0,0,0


W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,7,7,7,0,0,0,0,0,0,0
5,14,,2,0,0,0,0,0,0,0,0,0,0,0,0
1,8,,3,0,0,0,0,0,0,0,0,0,0,0,0
6,16,,4,0,0,0,0,0,0,0,0,0,0,0,0
7,2,,5,0,0,0,0,0,0,0,0,0,0,0,0


W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,7,7,7,7,0,0,0,0,0,0
5,14,,2,0,0,0,0,0,0,0,0,0,0,0,0
1,8,,3,0,0,0,0,0,0,0,0,0,0,0,0
6,16,,4,0,0,0,0,0,0,0,0,0,0,0,0
7,2,,5,0,0,0,0,0,0,0,0,0,0,0,0


...

W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,7,7,7,7,7,7,7,7,7,7
5,14,,2,0,0,7,7,7,14,14,21,21,21,21,21
1,8,,3,0,8,8,15,15,15,22,22,29,29,29,29
6,16,,4,0,8,8,15,15,15,22,24,29,31,31,31
7,2,,5,0,8,8,15,15,15,22,24,29,0,0,0


W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,7,7,7,7,7,7,7,7,7,7
5,14,,2,0,0,7,7,7,14,14,21,21,21,21,21
1,8,,3,0,8,8,15,15,15,22,22,29,29,29,29
6,16,,4,0,8,8,15,15,15,22,24,29,31,31,31
7,2,,5,0,8,8,15,15,15,22,24,29,31,0,0


W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,7,7,7,7,7,7,7,7,7,7
5,14,,2,0,0,7,7,7,14,14,21,21,21,21,21
1,8,,3,0,8,8,15,15,15,22,22,29,29,29,29
6,16,,4,0,8,8,15,15,15,22,24,29,31,31,31
7,2,,5,0,8,8,15,15,15,22,24,29,31,31,0


W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,7,7,7,7,7,7,7,7,7,7
5,14,,2,0,0,7,7,7,14,14,21,21,21,21,21
1,8,,3,0,8,8,15,15,15,22,22,29,29,29,29
6,16,,4,0,8,8,15,15,15,22,24,29,31,31,31
7,2,,5,0,8,8,15,15,15,22,24,29,31,31,31


W,V,,#,0,1,2,3,4,5,6,7,8,9,10,11
-,-,,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,,1,0,0,7,7,7,7,7,7,7,7,7,7
5,14,,2,0,0,7,7,7,14,14,21,21,21,21,21
1,8,,3,0,8,8,15,15,15,22,22,29,29,29,29
6,16,,4,0,8,8,15,15,15,22,24,29,31,31,31
7,2,,5,0,8,8,15,15,15,22,24,29,31,31,31


The final table above shows the method of obtaining the final answer:
1. Go to the last cell of the matrix
2. If the value in the cell above equals the value of the current cell, go up 1 row. If it doesn't, this means you are taking the current item in your row. After selecting the item, go up one row and go to the column: j = j - weight of object you selected.

The final solution for the maximum value is: 31

The final solution for the items taken is a binary list corresponding to objects $O: \{O_1, O_2, ... \}$ where 1 means selected and 0 means not selected. The list for objects 1, 3, 4 is represented as [1, 0, 1, 1, 0]


## Complexity

Time complexity: $O(n*w)$ where $n$ is the number of items and $w$ is the weight limit

- You must fill $O(n*w)$ cells which takes $O(1)$ work per cell

Space complexity: $O(n*w)$